In [11]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [12]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [13]:
with open('model.pickle_rf_2_2024_06', 'rb') as f:
    model = pickle.load(f)

In [14]:
import io
#getting the stop words
f_stopWords = io.open("StopWords_425.txt", mode="r", encoding="utf-16")
sw = []
df_StopWordsRemoval_Sinhala = pd.DataFrame(columns=['Post', 'Label'])
sinhala_stop_words = []
for x in f_stopWords:
  sw.append(x.split()[0])

In [15]:
vocab = pd.read_csv('vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [16]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['Post'])
    data["Post"] = data["Post"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["Post"] = data['Post'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["Post"] = data["Post"].apply(remove_punctuations)
    data["Post"] = data['Post'].str.replace('\d+', '', regex=True)
    data["Post"] = data["Post"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    return data["Post"]

In [17]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [18]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)

    if prediction == 1:
        return 'true'
    else:
        return 'false'

In [19]:
txt = "රනිල් වෙනුවෙන් ජනපති මැතිවරණයට සහය පැතීමට වජිර අබේවර්ධන, ඥානසාර හිමි හමුවෙයි?"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'true'